### START A H2O CLUSTER


In [19]:
%matplotlib inline
import h2o
#h2o.init()
h2o.init(ip="localhost", port=54332 , max_mem_size_GB = 16)

Checking whether there is an H2O instance running at http://localhost:54332. connected.


H2O cluster uptime:,10 mins 55 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.6
H2O cluster version age:,9 days
H2O cluster name:,H2O_from_python_HarshithaGS_u5mtfh
H2O cluster total nodes:,1
H2O cluster free memory:,4.638 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


### IMPORT THE DATASET

path ---> The complete URL or normalized file path of the file to be imported. Each row of data appears as one line of the file.

destination_frame ---> (Optional) The unique hex key assigned to the imported file. If none is given, a key will automatically be generated based on the URL path. parse

In [20]:
destination_frame_name = 'lending_club_data.hex'
lc_loan_data = h2o.import_file(path=r'C:\Users\HarshithaGS\Desktop\DS- Assignment 3\Code\cl1.csv', destination_frame=destination_frame_name)

Parse progress: |█████████████████████████████████████████████████████████| 100%


# Encode response variable

Since we want to train a binary classification model, we must ensure that the response is coded as a factor. If the response is 0/1, H2O will assume it's numeric, which means that H2O will train a regression model instead.

In [21]:
lc_loan_data['int_rate'] = lc_loan_data['int_rate'].ascharacter()

lc_loan_data['int_rate'] = lc_loan_data['int_rate'].asfactor()  #encode the binary repsonse as a factor
lc_loan_data['int_rate'].levels()  #optional: after encoding, this shows the two factor levels, '0' and '1'

[['10',
  '10.01',
  '10.08',
  '10.14',
  '10.15',
  '10.16',
  '10.2',
  '10.25',
  '10.28',
  '10.33',
  '10.36',
  '10.37',
  '10.38',
  '10.39',
  '10.46',
  '10.49',
  '10.51',
  '10.59',
  '10.62',
  '10.64',
  '10.65',
  '10.71',
  '10.74',
  '10.75',
  '10.78',
  '10.83',
  '10.91',
  '10.95',
  '10.96',
  '10.99',
  '11.03',
  '11.09',
  '11.11',
  '11.12',
  '11.14',
  '11.22',
  '11.26',
  '11.28',
  '11.34',
  '11.36',
  '11.41',
  '11.44',
  '11.46',
  '11.48',
  '11.49',
  '11.53',
  '11.54',
  '11.55',
  '11.58',
  '11.59',
  '11.63',
  '11.66',
  '11.67',
  '11.71',
  '11.72',
  '11.78',
  '11.83',
  '11.86',
  '11.89',
  '11.91',
  '11.97',
  '11.99',
  '12.04',
  '12.05',
  '12.09',
  '12.12',
  '12.17',
  '12.18',
  '12.21',
  '12.22',
  '12.23',
  '12.29',
  '12.35',
  '12.36',
  '12.39',
  '12.41',
  '12.42',
  '12.49',
  '12.53',
  '12.54',
  '12.59',
  '12.61',
  '12.62',
  '12.67',
  '12.68',
  '12.69',
  '12.72',
  '12.73',
  '12.8',
  '12.84',
  '12.85',
  '1

# Partition data

Next, we partition the data into training, validation and test sets.

In [22]:
# Partition data into 70%, 15%, 15% chunks
# Setting a seed will guarantee reproducibility

splits = lc_loan_data.split_frame(ratios=[0.7, 0.15], seed=5)  

train = splits[0]
valid = splits[1]
test = splits[2]

In [23]:
train.head()

C1,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_int,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
1,1.07743e+06,2500,2500,2500,60,15.27,59.83,3,10,6,30000,2,22,1,1,11,1,0,5,0,0,3,0,1687,9.4,4,0,0,1008.71,1008.71,435.17,117.08,1.11,119.66,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.07686e+06,10000,10000,10000,36,13.49,339.31,3,10,6,49200,2,22,6,10,5,20,0,1,35,0,10,0,5598,21,37,0,0,12226.3,12226.3,2209.33,0,0,357.48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.07536e+06,3000,3000,3000,60,12.69,67.79,2,1,6,80000,2,22,2,10,38,17.94,0,0,38,0,15,0,27783,53.9,38,766.9,766.9,3242.17,3242.17,1009.07,0,0,67.79,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1.07205e+06,3000,3000,3000,36,18.64,109.43,5,9,6,48000,2,22,6,1,5,5.35,0,2,0,0,4,0,8221,87.5,4,0,0,3938.14,3938.14,938.14,0,0,111.34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1.07157e+06,5375,5375,5350,60,12.69,121.45,2,10,6,15000,3,22,1,10,44,18.08,0,0,0,0,2,0,9279,36.5,3,0,0,1476.19,1469.34,533.42,269.29,2.52,121.45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,1.07008e+06,6500,6500,6500,60,14.65,153.45,3,5,5,72000,1,22,6,3,4,16.12,0,2,0,0,14,0,4032,20.6,23,0,0,7677.52,7677.52,1177.52,0,0,1655.54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11,1.06991e+06,12000,12000,12000,36,12.69,402.54,2,10,5,75000,2,22,6,3,5,10.78,0,0,0,0,12,0,23336,67.1,34,0,0,13943.1,13943.1,1943.08,0,0,6315.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,1.06469e+06,9000,9000,9000,36,13.49,305.38,3,10,6,30000,2,22,1,3,46,10.08,0,1,0,0,4,0,10452,91.7,9,0,0,2270.7,2270.7,570.26,444.3,4.16,305.38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,1.06987e+06,3000,3000,3000,36,9.91,96.68,2,3,6,15000,2,22,6,2,15,12.56,0,2,0,0,11,0,7323,43.1,11,0,0,3478.98,3478.98,478.98,0,0,102.43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,1.06906e+06,10000,10000,10000,36,10.65,325.74,2,3,6,100000,2,22,1,10,5,7.06,0,2,0,0,14,0,11997,55.5,29,0,0,7471.99,7471.99,1393.42,645.1,6.3145,325.74,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Notice that split_frame() uses approximate splitting not exact splitting (for efficiency), so these are not exactly 70%, 15% and 15% of the total rows.

In [24]:
print (train.nrow)
print (valid.nrow)
print (test.nrow)

621334
133178
132867


# Identify response and predictor variables

In H2O, we use y to designate the response variable and x to designate the list of predictor columns

In [25]:
y = 'int_rate'
x = list(lc_loan_data.columns)

In [26]:
x.remove(y)  #remove the response

# H2O MACHINE LEARNING


Now that we have prepared the data, we can train some models. We will start by training a single model from each of the H2O supervised algos:

  1. Generalized Linear Model (GLM)
  2. Random Forest (RF)
  3. Deep Learning (DL)

1. GENERALIZED LINEAR MODEL

Let's start with a basic binomial Generalized Linear Model (GLM). By default, H2O's GLM uses a regularized, elastic net model.

In [27]:
# Import H2O GLM:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

Create an object of class, "H2OGeneralizedLinearEstimator". This does not  do any training, it just sets the model up for training by specifying model parameters.

In [28]:
# Initialize the GLM estimator:
glm_fit = H2OGeneralizedLinearEstimator(family = 'ordinal' ,model_id='glm_fit1')

Now that glm_fit object is initialized, we can train the model:

In [29]:
glm_fit.train(x=x, y=y, training_frame=train)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [30]:
glm_fit

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  glm_fit1


ModelMetricsOrdinalGLM: glm
** Reported on train data. **

MSE: 0.9998085421396531
RMSE: 0.9999042664873738
Scoring History: 


,timestamp,duration,iterations,negative_log_likelihood,objective
,2019-03-22 21:31:35,0.000 sec,0,120159576.0424282,193.3896681


In [31]:
y_pred=glm_fit.predict(test)

glm prediction progress: |████████████████████████████████████████████████| 100%


In [32]:
y_pred

predict p10 10.01 10.08 10.14 10.15 10.16 10.2 10.25 10.28 10.33 10.36 10.37 10.38 10.39 10.46 10.49 10.51 10.59 10.62 10.64 10.65 10.71 10.74 10.75 10.78 10.83 10.91 10.95 10.96 10.99 11.03 11.09 11.11 11.12 11.14 11.22 11.26 11.28 11.34 11.36 11.41 11.44 11.46 11.48 11.49 11.53 11.54 11.55 11.58 11.59 11.63 11.66 11.67 11.71 11.72 11.78 11.83 11.86 11.89 11.91 11.97 11.99 12.04 12.05 12.09 12.12 12.17 12.18 12.21 12.22 12.23 12.29 12.35 12.36 12.39 12.41 12.42 12.49 12.53 12.54 12.59 12.61 12.62 12.67 12.68 12.69 12.72 12.73 12.8 12.84 12.85 12.86 12.87 12.88 12.92 12.98 12.99 13.04 13.05 13.06 13.11 13.12 13.16 13.17 13.18 13.22 13.23 13.24 13.3 13.33 13.35 13.36 13.43 13.44 13.47 13.48 13.49 13.53 13.55 13.57 13.61 13.62 13.65 13.66 13.67 13.68 13.72 13.75 13.79 13.8 13.84 13.85 13.87 13.92 13.93 13.98 13.99 14.07 14.09 14.11 14.12 14.16 14.17 14.18 14.22 14.25 14.26 14.27 14.28 14.3 14.31 14.33 14.35 14.38 14.42 14.43 14.46 14.47 14.48 14.49 14.5 14.54 14.57 14.59 14.61 14.62 14.64 14.65 14.67 14.7 14.72 14.74 14.75 14.79 14.82 14.83 14.84 14.85 14.88 14.91 14.93 14.96 14.98 14.99 15.01 15.05 15.07 15.1 15.13 15.2 15.21 15.22 15.23 15.25 15.27 15.28 15.31 15.33 15.37 15.38 15.41 15.45 15.51 15.57 15.58 15.59 15.61 15.62 15.65 15.68 15.7 15.76 15.77 15.8 15.81 15.83 15.88 15.95 15.96 15.99 p16 16.01 16.02 16.07 16.08 16.11 16.15 16.2 16.24 16.28 16.29 16.32 16.33 16.35 16.4 16.45 16.46 16.49 16.55 16.59 16.63 16.65 16.69 16.7 16.71 16.77 16.78 16.82 16.83 16.89 16.91 16.95 16.96 16.99 17.03 17.04 17.06 17.09 17.1 17.14 17.15 17.19 17.22 17.26 17.27 17.28 17.34 17.39 17.41 17.43 17.44 17.46 17.49 17.51 17.54 17.56 17.57 17.58 17.59 17.66 17.72 17.74 17.76 17.77 17.78 17.8 17.86 17.88 17.9 17.91 17.93 17.97 17.99 18.04 18.07 18.09 18.17 18.2 18.21 18.24 18.25 18.29 18.3 18.36 18.39 18.43 18.49 18.53 18.54 18.55 18.61 18.62 18.64 18.67 18.72 18.75 18.78 18.79 18.84 18.85 18.86 18.91 18.92 18.99 19.03 19.04 19.05 19.13 19.16 19.19 19.2 19.22 19.24 19.29 19.36 19.39 19.41 19.42 19.47 19.48 19.52 19.66 19.69 19.72 19.74 19.79 19.82 19.89 19.91 19.97 19.99 20.03 20.11 20.16 20.17 20.2 20.25 20.3 20.31 20.4 20.48 20.49 20.5 20.52 20.53 20.62 20.69 20.77 20.8 20.85 20.86 20.89 20.9 20.99 p21 21.14 21.15 21.18 21.21 21.22 21.27 21.28 21.36 21.48 21.49 21.59 21.6 21.64 21.67 21.7 21.74 21.82 21.97 21.98 21.99 22.06 22.11 22.15 22.2 22.35 22.4 22.45 22.47 22.48 22.64 22.7 22.74 22.78 22.85 22.9 22.94 22.95 22.99 23.1 23.13 23.22 23.26 23.28 23.33 23.4 23.43 23.5 23.52 23.59 23.63 23.7 23.76 23.83 23.91 23.99 24.08 24.11 24.2 24.24 24.33 24.4 24.5 24.52 24.59 24.7 24.76 24.83 24.89 24.99 25.09 25.28 25.57 25.78 25.8 25.83 25.89 25.99 26.06 26.77 26.99 27.31 27.49 27.88 27.99 28.49 28.99 5.32 5.42 5.79 5.93 5.99 p6 6.03 6.17 6.24 6.39 6.49 6.54 6.62 6.68 6.76 6.89 6.91 6.92 6.97 6.99 7.05 7.12 7.14 7.26 7.29 7.37 7.4 7.42 7.43 7.49 7.51 7.62 7.66 7.68 7.69 7.74 7.75 7.88 7.89 7.9 7.91 p8 8.07 8.18 8.19 8.32 8.38 8.39 8.49 8.59 8.6 8.63 8.67 8.7 8.88 8.9 8.94 9.01 9.07 9.17 9.2 9.25 9.32 9.33 9.38 9.45 9.49 9.51 9.62 9.63 9.64 9.67 9.7 9.71 9.76 9.8 9.83 9.88 9.91 9.96 9.99 
 
 
 17.49 4.59945e-235 1.88502e-234 1.82541e-234 1.81292e-233 2.28982e-233 5.4358e-234 1.77116e-231 3.32209e-232 8.48518e-231 6.51489e-231 2.76481e-230 1.85748e-228 1.45971e-228 3.67747e-229 1.17908e-227 1.01564e-227 2.05963e-227 1.28807e-226 5.11995e-224 3.58949e-224 8.96579e-225 8.24864e-221 3.84664e-220 1.6437e-219 4.50988e-216 5.57209e-214 6.50084e-214 3.35091e-214 1.46941e-211 6.3016e-211 2.98999e-210 2.14108e-210 2.38457e-208 5.31137e-208 1.04336e-207 2.20956e-207 2.88744e-207 1.58897e-206 6.0051e-207 1.93132e-204 2.1082e-203 1.52777e-203 6.11875e-203 1.53593e-202 6.22135e-201 4.3838e-201 8.42641e-200 4.06486e-199 8.54484e-200 3.82337e-199 3.23484e-196 1.82821e-195 2.76036e-196 3.18605e-194 1.11704e-195 5.05908e-193 7.81466e-194 4.17979e-193 1.14851e-191 5.2751e-192 2.2231e-190 1.79854e-190 1.17679e-189 3.5258e-189 1.14427e-189 7.50651e-189 4.34677e-188 3.4462

2. RANDOM FOREST

H2O's Random Forest (RF) is implemented as a distributed version of the standard Random Forest algorithm and variable importance measures.

In [33]:
# Import H2O RF:
from h2o.estimators.random_forest import H2ORandomForestEstimator

First we  train a basic Random Forest model with default parameters.
Random Forest will infer the response distribution from the response encoding.
A seed is required for reproducibility.

In [34]:
# Initialize the RF estimator:

rf_fit = H2ORandomForestEstimator(model_id='rf_fit', seed=1)

In [35]:
train.head

C1,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_int,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
1,1.07743e+06,2500,2500,2500,60,15.27,59.83,3,10,6,30000,2,22,1,1,11,1,0,5,0,0,3,0,1687,9.4,4,0,0,1008.71,1008.71,435.17,117.08,1.11,119.66,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.07686e+06,10000,10000,10000,36,13.49,339.31,3,10,6,49200,2,22,6,10,5,20,0,1,35,0,10,0,5598,21,37,0,0,12226.3,12226.3,2209.33,0,0,357.48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.07536e+06,3000,3000,3000,60,12.69,67.79,2,1,6,80000,2,22,2,10,38,17.94,0,0,38,0,15,0,27783,53.9,38,766.9,766.9,3242.17,3242.17,1009.07,0,0,67.79,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1.07205e+06,3000,3000,3000,36,18.64,109.43,5,9,6,48000,2,22,6,1,5,5.35,0,2,0,0,4,0,8221,87.5,4,0,0,3938.14,3938.14,938.14,0,0,111.34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1.07157e+06,5375,5375,5350,60,12.69,121.45,2,10,6,15000,3,22,1,10,44,18.08,0,0,0,0,2,0,9279,36.5,3,0,0,1476.19,1469.34,533.42,269.29,2.52,121.45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,1.07008e+06,6500,6500,6500,60,14.65,153.45,3,5,5,72000,1,22,6,3,4,16.12,0,2,0,0,14,0,4032,20.6,23,0,0,7677.52,7677.52,1177.52,0,0,1655.54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11,1.06991e+06,12000,12000,12000,36,12.69,402.54,2,10,5,75000,2,22,6,3,5,10.78,0,0,0,0,12,0,23336,67.1,34,0,0,13943.1,13943.1,1943.08,0,0,6315.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,1.06469e+06,9000,9000,9000,36,13.49,305.38,3,10,6,30000,2,22,1,3,46,10.08,0,1,0,0,4,0,10452,91.7,9,0,0,2270.7,2270.7,570.26,444.3,4.16,305.38,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,1.06987e+06,3000,3000,3000,36,9.91,96.68,2,3,6,15000,2,22,6,2,15,12.56,0,2,0,0,11,0,7323,43.1,11,0,0,3478.98,3478.98,478.98,0,0,102.43,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,1.06906e+06,10000,10000,10000,36,10.65,325.74,2,3,6,100000,2,22,1,10,5,7.06,0,2,0,0,14,0,11997,55.5,29,0,0,7471.99,7471.99,1393.42,645.1,6.3145,325.74,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<bound method H2OFrame.head of >

Now that rf_fit object is initialized, we can train the model:

In [36]:
rf_fit.train(x=x, y=y, training_frame=train)

drf Model Build progress: |

H2OServerError: HTTP 500 Server Error:
'<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html;charset=ISO-8859-1"/>\n<title>Error 500 Server Error</title>\n</head>\n<body>\n<h2>HTTP ERROR: 500</h2>\n<p>Problem accessing /3/Jobs/$03017f0000013dd4ffffffff$_9787d13058986f3fdf1455197494495e. Reason:\n<pre>    Server Error</pre></p>\n\n                                                \n                                                \n                                                \n                                                \n                                                \n                                                \n                                                \n                                                \n                                                \n                                                \n                                                \n                                                \n                                                \n    

3. DEEP LEARNING (DL)

H2O's Deep Learning algorithm is a multilayer feed-forward artificial neural network. It can also be used to train an autoencoder, however, in the example below we will train a standard supervised prediction model.

In [ ]:
# Import H2O DL:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

The DL infers the response distribution from the response encoding if not specified explicitly through the distribution argument.
H2O's DL will not be reproducbible if run on more than a single core, so in this example, the performance metrics below may vary slightly.

In [ ]:
# Initialize and train the DL estimator:

dl_fit = H2ODeepLearningEstimator(model_id='dl_fit', seed=1)
dl_fit.train(x=x, y=y, training_frame=train)

# EVALUATE MODEL PERFORMANCE

Comparing  the performance of the GLM , RF , DL models that were just trained.

In [ ]:
perf1 = glm_fit.model_performance(test)
#perf2 = rf_fit.model_performance(test)
perf3 = dl_fit.model_performance(test)

In [ ]:
str(perf1)
str(perf3)

# Retreive test set AUC(Area under the curve)
#print (perf1.auc())
#print (perf2.auc())
#print (perf3.auc())

In [ ]:
h2o.cluster().shutdown(prompt=False)